In [1]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib

list1 = []
list2 = []

with open('img.txt', 'r') as f:
    for line in f:
        list1.append(line.strip())

    f.close()

with open('label.txt', 'r') as p:
    for line in p:
        list2.append(line.strip())

    p.close()

class Dataset(object):
    def __init__(self, root, transform):
        self.root = root
        self.transforms = transform
        
        # load all image files, sorting them to
        # ensure that they are aligned
        
#         self.imgs = (self.imgs - min(self.imgs))/(max(self.imgs) - min(self.imgs))
        
        self.imgs = list1
        self.masks = list2
        #self.masks = mat2gray(list2)
#         self.imgs = list(sorted(os.listdir(os.path.join(root, "img"))))
#         self.masks = list(sorted(os.listdir(os.path.join(root, "label"))))
       
            


    def __getitem__(self, idx):
        # load images ad masks
#         img_path= os.path.join("Train_set/img/",self.imgs[idx])
#         mask_path= os.path.join("Train_set/label/",self.masks[idx])
        img_path = os.path.join(self.root, "img", self.imgs[idx])
        mask_path = os.path.join(self.root, "label", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        
        #img = (img - min(img))/(max(img) - min(img))
        
        #img = img.resize((224,224))
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        #Resize = transforms.Resize((224,224), interpolation=Image.NEAREST)
        
        mask = Image.open(mask_path)
        
        #mask = Resize(mask)
        #print("mask path", mask_path)
    
        #mask = im2double(mask)
        #mask = mask.resize((224,224))
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]
        
        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        #print(boxes.shape)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
         
        if self.transforms is not None:
            img, target = self.transforms(img, target)
    
        return img, target
        
        

    def __len__(self):
        return len(self.imgs)

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


In [3]:
from engine  import train_one_epoch, evaluate
import utils
import torch
import torchvision
import torchvision.transforms as transforms
import dataset
import argparse
import parser
import time

# 학습을 GPU로 진행하되 GPU가 가용하지 않으면 CPU로 합니다


#parser.add_argument("--gpu", type=str, default="7", help="gpu ID number to use.")      
#args = parser.parse_args()    
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"    
os.environ["CUDA_VISIBLE_DEVICES"] = "1"    
torch.backends.cudnn.benchmark = True    
torch.backends.cudnn.enabled = True  
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 우리 데이터셋은 두 개의 클래스만 가집니다 - 배경과 사람
num_classes = 256
# 데이터셋과 정의된 변환들을 사용합니다

In [4]:

dataset = Dataset('Train_set', get_transform(train=True))
dataset_test = Dataset('Train_set', get_transform(train=False))


#transform normalize
#     dataset_mean = dataset.data.mean(axis=(0,1,2))
#     dataset_std = dataset.data.std(axis=(0,1,2))

#     dataset_mean = dataset_mean/255
#     dataset_std = dataset_mean/255

#     transform_train = transforms.Compose([
#         transforms.ToTensor(),
#         Transforms,Normalize(dataset_mean,dataset_std)
#     ])
# 데이터셋을 학습용과 테스트용으로 나눕니다(역자주: 여기서는 전체의 50개를 테스트에, 나머지를 학습에 사용합니다)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])


# 데이터 로더를 학습용과 검증용으로 정의합니다
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [5]:
# 도움 함수를 이용해 모델을 가져옵니다
model = get_model_instance_segmentation(num_classes)

# 모델을 GPU나 CPU로 옮깁니다
model.to(device)

# 옵티마이저(Optimizer)를 만듭니다
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0001,
                            momentum=0.9, weight_decay=0.0005)

# 학습률 스케쥴러를 만듭니다
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

#     # 10 에포크만큼 학습해봅시다
#     num_epochs = 10

#     for epoch in range(num_epochs):
#         # 1 에포크동안 학습하고, 10회 마다 출력합니다
#         train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

#         # 학습률을 업데이트 합니다
#         lr_scheduler.step()
#         # 테스트 데이터셋에서 평가를 합니다
#         evaluate(model, data_loader_test, device=device)
#         #time.strftime('Now : %c', time.localtime(time.time()))
#         torch.save(model.state_dict(), 'epoch_{}.tar'.format(epoch))
#     print("model has been saved successfully!")


In [6]:
import torchvision
from torchvision.utils import save_image

In [7]:
os.mkdir("Original_OCID")
os.mkdir("Segmentation_OCID")

In [8]:
from PIL import Image
import torch
import transforms as T
from matplotlib import pyplot as plt
num_epochs = 10
#get one image from test img.

for i in range(len(dataset_test)):
    img, _ = dataset_test[i]
    org=Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    os.mkdir("Segmentation_OCID/data_{}".format(i))
    
    for epoch in range(num_epochs):
        
        os.mkdir("Segmentation_OCID/data_{}/epoch_{}".format(i,epoch))
        num_classes = 256
        model = get_model_instance_segmentation(
                num_classes)
        model.cuda()
        model.load_state_dict(torch.load("epoch_{}.tar".format(epoch)))
        model.eval()
        with torch.no_grad():
            prediction = model([img.to(device)])
    #         print(image.shape)
            for images in range(len(prediction[-1]['masks'])):
                try:
                    
                    im=Image.fromarray(prediction[-1]['masks'][images, 0].mul(255).byte().cpu().numpy())
                    #print(len(prediction[-1]['masks']))
                    print("classes =",images)
                    print("epochs =",epoch)
                    print("dataset = ",i)
                    #display(im)
                    im.save("Segmentation_OCID/data_{}/epoch_{}/data{}_epoch{}_index{}.png".format(i,epoch,i,epoch,images))
                    org.save("Original_OCID/data_{}.png".format(i))
                except:
                    print("Error")


classes = 0
epochs = 0
dataset =  0
classes = 1
epochs = 0
dataset =  0
classes = 2
epochs = 0
dataset =  0
classes = 3
epochs = 0
dataset =  0
classes = 4
epochs = 0
dataset =  0
classes = 5
epochs = 0
dataset =  0
classes = 6
epochs = 0
dataset =  0
classes = 7
epochs = 0
dataset =  0
classes = 8
epochs = 0
dataset =  0
classes = 9
epochs = 0
dataset =  0
classes = 10
epochs = 0
dataset =  0
classes = 11
epochs = 0
dataset =  0
classes = 12
epochs = 0
dataset =  0
classes = 13
epochs = 0
dataset =  0
classes = 14
epochs = 0
dataset =  0
classes = 15
epochs = 0
dataset =  0
classes = 16
epochs = 0
dataset =  0
classes = 17
epochs = 0
dataset =  0
classes = 18
epochs = 0
dataset =  0
classes = 19
epochs = 0
dataset =  0
classes = 20
epochs = 0
dataset =  0
classes = 21
epochs = 0
dataset =  0
classes = 22
epochs = 0
dataset =  0
classes = 23
epochs = 0
dataset =  0
classes = 24
epochs = 0
dataset =  0
classes = 25
epochs = 0
dataset =  0
classes = 26
epochs = 0
dataset =  0
classes = 2

classes = 32
epochs = 5
dataset =  0
classes = 33
epochs = 5
dataset =  0
classes = 34
epochs = 5
dataset =  0
classes = 35
epochs = 5
dataset =  0
classes = 36
epochs = 5
dataset =  0
classes = 37
epochs = 5
dataset =  0
classes = 38
epochs = 5
dataset =  0
classes = 39
epochs = 5
dataset =  0
classes = 0
epochs = 6
dataset =  0
classes = 1
epochs = 6
dataset =  0
classes = 2
epochs = 6
dataset =  0
classes = 3
epochs = 6
dataset =  0
classes = 4
epochs = 6
dataset =  0
classes = 5
epochs = 6
dataset =  0
classes = 6
epochs = 6
dataset =  0
classes = 7
epochs = 6
dataset =  0
classes = 8
epochs = 6
dataset =  0
classes = 9
epochs = 6
dataset =  0
classes = 10
epochs = 6
dataset =  0
classes = 11
epochs = 6
dataset =  0
classes = 12
epochs = 6
dataset =  0
classes = 13
epochs = 6
dataset =  0
classes = 14
epochs = 6
dataset =  0
classes = 15
epochs = 6
dataset =  0
classes = 16
epochs = 6
dataset =  0
classes = 17
epochs = 6
dataset =  0
classes = 18
epochs = 6
dataset =  0
classes = 1

classes = 16
epochs = 1
dataset =  1
classes = 17
epochs = 1
dataset =  1
classes = 18
epochs = 1
dataset =  1
classes = 19
epochs = 1
dataset =  1
classes = 20
epochs = 1
dataset =  1
classes = 21
epochs = 1
dataset =  1
classes = 22
epochs = 1
dataset =  1
classes = 23
epochs = 1
dataset =  1
classes = 24
epochs = 1
dataset =  1
classes = 25
epochs = 1
dataset =  1
classes = 26
epochs = 1
dataset =  1
classes = 27
epochs = 1
dataset =  1
classes = 28
epochs = 1
dataset =  1
classes = 29
epochs = 1
dataset =  1
classes = 30
epochs = 1
dataset =  1
classes = 31
epochs = 1
dataset =  1
classes = 32
epochs = 1
dataset =  1
classes = 33
epochs = 1
dataset =  1
classes = 34
epochs = 1
dataset =  1
classes = 35
epochs = 1
dataset =  1
classes = 36
epochs = 1
dataset =  1
classes = 37
epochs = 1
dataset =  1
classes = 38
epochs = 1
dataset =  1
classes = 39
epochs = 1
dataset =  1
classes = 0
epochs = 2
dataset =  1
classes = 1
epochs = 2
dataset =  1
classes = 2
epochs = 2
dataset =  1
clas

classes = 26
epochs = 6
dataset =  1
classes = 27
epochs = 6
dataset =  1
classes = 28
epochs = 6
dataset =  1
classes = 29
epochs = 6
dataset =  1
classes = 30
epochs = 6
dataset =  1
classes = 31
epochs = 6
dataset =  1
classes = 32
epochs = 6
dataset =  1
classes = 33
epochs = 6
dataset =  1
classes = 34
epochs = 6
dataset =  1
classes = 35
epochs = 6
dataset =  1
classes = 36
epochs = 6
dataset =  1
classes = 37
epochs = 6
dataset =  1
classes = 38
epochs = 6
dataset =  1
classes = 39
epochs = 6
dataset =  1
classes = 0
epochs = 7
dataset =  1
classes = 1
epochs = 7
dataset =  1
classes = 2
epochs = 7
dataset =  1
classes = 3
epochs = 7
dataset =  1
classes = 4
epochs = 7
dataset =  1
classes = 5
epochs = 7
dataset =  1
classes = 6
epochs = 7
dataset =  1
classes = 7
epochs = 7
dataset =  1
classes = 8
epochs = 7
dataset =  1
classes = 9
epochs = 7
dataset =  1
classes = 10
epochs = 7
dataset =  1
classes = 11
epochs = 7
dataset =  1
classes = 12
epochs = 7
dataset =  1
classes = 1

classes = 0
epochs = 2
dataset =  2
classes = 1
epochs = 2
dataset =  2
classes = 2
epochs = 2
dataset =  2
classes = 3
epochs = 2
dataset =  2
classes = 4
epochs = 2
dataset =  2
classes = 5
epochs = 2
dataset =  2
classes = 6
epochs = 2
dataset =  2
classes = 7
epochs = 2
dataset =  2
classes = 8
epochs = 2
dataset =  2
classes = 9
epochs = 2
dataset =  2
classes = 10
epochs = 2
dataset =  2
classes = 11
epochs = 2
dataset =  2
classes = 12
epochs = 2
dataset =  2
classes = 13
epochs = 2
dataset =  2
classes = 14
epochs = 2
dataset =  2
classes = 15
epochs = 2
dataset =  2
classes = 16
epochs = 2
dataset =  2
classes = 17
epochs = 2
dataset =  2
classes = 18
epochs = 2
dataset =  2
classes = 19
epochs = 2
dataset =  2
classes = 20
epochs = 2
dataset =  2
classes = 21
epochs = 2
dataset =  2
classes = 22
epochs = 2
dataset =  2
classes = 23
epochs = 2
dataset =  2
classes = 24
epochs = 2
dataset =  2
classes = 25
epochs = 2
dataset =  2
classes = 26
epochs = 2
dataset =  2
classes = 2

...

classes = 6
epochs = 1
dataset =  41
classes = 7
epochs = 1
dataset =  41
classes = 8
epochs = 1
dataset =  41
classes = 9
epochs = 1
dataset =  41
classes = 10
epochs = 1
dataset =  41
classes = 11
epochs = 1
dataset =  41
classes = 12
epochs = 1
dataset =  41
classes = 13
epochs = 1
dataset =  41
classes = 14
epochs = 1
dataset =  41
classes = 15
epochs = 1
dataset =  41
classes = 16
epochs = 1
dataset =  41
classes = 17
epochs = 1
dataset =  41
classes = 18
epochs = 1
dataset =  41
classes = 19
epochs = 1
dataset =  41
classes = 20
epochs = 1
dataset =  41
classes = 21
epochs = 1
dataset =  41
classes = 22
epochs = 1
dataset =  41
classes = 23
epochs = 1
dataset =  41
classes = 24
epochs = 1
dataset =  41
classes = 25
epochs = 1
dataset =  41
classes = 26
epochs = 1
dataset =  41
classes = 0
epochs = 2
dataset =  41
classes = 1
epochs = 2
dataset =  41
classes = 2
epochs = 2
dataset =  41
classes = 3
epochs = 2
dataset =  41
classes = 4
epochs = 2
dataset =  41
classes = 5
epochs = 

classes = 6
epochs = 9
dataset =  41
classes = 7
epochs = 9
dataset =  41
classes = 8
epochs = 9
dataset =  41
classes = 9
epochs = 9
dataset =  41
classes = 10
epochs = 9
dataset =  41
classes = 11
epochs = 9
dataset =  41
classes = 12
epochs = 9
dataset =  41
classes = 13
epochs = 9
dataset =  41
classes = 14
epochs = 9
dataset =  41
classes = 15
epochs = 9
dataset =  41
classes = 16
epochs = 9
dataset =  41
classes = 17
epochs = 9
dataset =  41
classes = 18
epochs = 9
dataset =  41
classes = 19
epochs = 9
dataset =  41
classes = 20
epochs = 9
dataset =  41
classes = 21
epochs = 9
dataset =  41
classes = 22
epochs = 9
dataset =  41
classes = 23
epochs = 9
dataset =  41
classes = 24
epochs = 9
dataset =  41
classes = 25
epochs = 9
dataset =  41
classes = 26
epochs = 9
dataset =  41
classes = 0
epochs = 0
dataset =  42
classes = 1
epochs = 0
dataset =  42
classes = 2
epochs = 0
dataset =  42
classes = 3
epochs = 0
dataset =  42
classes = 4
epochs = 0
dataset =  42
classes = 5
epochs = 

classes = 6
epochs = 3
dataset =  46
classes = 7
epochs = 3
dataset =  46
classes = 8
epochs = 3
dataset =  46
classes = 9
epochs = 3
dataset =  46
classes = 10
epochs = 3
dataset =  46
classes = 11
epochs = 3
dataset =  46
classes = 12
epochs = 3
dataset =  46
classes = 13
epochs = 3
dataset =  46
classes = 14
epochs = 3
dataset =  46
classes = 15
epochs = 3
dataset =  46
classes = 16
epochs = 3
dataset =  46
classes = 17
epochs = 3
dataset =  46
classes = 18
epochs = 3
dataset =  46
classes = 19
epochs = 3
dataset =  46
classes = 20
epochs = 3
dataset =  46
classes = 21
epochs = 3
dataset =  46
classes = 22
epochs = 3
dataset =  46
classes = 0
epochs = 4
dataset =  46
classes = 1
epochs = 4
dataset =  46
classes = 2
epochs = 4
dataset =  46
classes = 3
epochs = 4
dataset =  46
classes = 4
epochs = 4
dataset =  46
classes = 5
epochs = 4
dataset =  46
classes = 6
epochs = 4
dataset =  46
classes = 7
epochs = 4
dataset =  46
classes = 8
epochs = 4
dataset =  46
classes = 9
epochs = 4
da

classes = 8
epochs = 5
dataset =  48
classes = 9
epochs = 5
dataset =  48
classes = 0
epochs = 6
dataset =  48
classes = 1
epochs = 6
dataset =  48
classes = 2
epochs = 6
dataset =  48
classes = 3
epochs = 6
dataset =  48
classes = 4
epochs = 6
dataset =  48
classes = 5
epochs = 6
dataset =  48
classes = 6
epochs = 6
dataset =  48
classes = 7
epochs = 6
dataset =  48
classes = 8
epochs = 6
dataset =  48
classes = 9
epochs = 6
dataset =  48
classes = 0
epochs = 7
dataset =  48
classes = 1
epochs = 7
dataset =  48
classes = 2
epochs = 7
dataset =  48
classes = 3
epochs = 7
dataset =  48
classes = 4
epochs = 7
dataset =  48
classes = 5
epochs = 7
dataset =  48
classes = 6
epochs = 7
dataset =  48
classes = 7
epochs = 7
dataset =  48
classes = 8
epochs = 7
dataset =  48
classes = 9
epochs = 7
dataset =  48
classes = 0
epochs = 8
dataset =  48
classes = 1
epochs = 8
dataset =  48
classes = 2
epochs = 8
dataset =  48
classes = 3
epochs = 8
dataset =  48
classes = 4
epochs = 8
dataset =  48
c

classes = 16
epochs = 5
dataset =  49
classes = 17
epochs = 5
dataset =  49
classes = 18
epochs = 5
dataset =  49
classes = 19
epochs = 5
dataset =  49
classes = 20
epochs = 5
dataset =  49
classes = 21
epochs = 5
dataset =  49
classes = 22
epochs = 5
dataset =  49
classes = 23
epochs = 5
dataset =  49
classes = 24
epochs = 5
dataset =  49
classes = 25
epochs = 5
dataset =  49
classes = 26
epochs = 5
dataset =  49
classes = 27
epochs = 5
dataset =  49
classes = 28
epochs = 5
dataset =  49
classes = 29
epochs = 5
dataset =  49
classes = 30
epochs = 5
dataset =  49
classes = 31
epochs = 5
dataset =  49
classes = 32
epochs = 5
dataset =  49
classes = 33
epochs = 5
dataset =  49
classes = 34
epochs = 5
dataset =  49
classes = 35
epochs = 5
dataset =  49
classes = 0
epochs = 6
dataset =  49
classes = 1
epochs = 6
dataset =  49
classes = 2
epochs = 6
dataset =  49
classes = 3
epochs = 6
dataset =  49
classes = 4
epochs = 6
dataset =  49
classes = 5
epochs = 6
dataset =  49
classes = 6
epochs

In [41]:
len(dataset_test)

50